### Author: Carmen Alonso Martínez

# Gráficos geoespaciales: mapas

Habitualmente necesitamos pintar datos que tienen una referencia geoespacial, por ejemplo:

* Puntos en mapas
* Rutas
* Estados, provincias, países u otras divisiones administrativas de territorios

Para hacerlo, podemos utilizar la librería `folium`, que permite utilizar [`leaflet`](https://leafletjs.com/) dentro de python.

Puedes consultar la documentación de folium [aquí](https://python-visualization.github.io/folium/).

In [1]:
#!pip install folium==0.11.0

In [3]:
import folium
import pandas as pd

In [4]:
print(folium.__version__)

0.11.0


Si te falla el `import folium` es que no tienes la librería instalada. Para hacerlo, corre desde tu terminal:

```
# si usas conda (gestor de paquetes de Anaconda)
conda install -c conda-forge folium

# si usas pip
pip install folium
```

Una vez lo instalas, tendrás que reiniciar el kernel de jupyter antes de volver a probar. Puedes hacerlo desde este notebook en el menú Kernel / Restart.

## Un primer mapa

Para pintar un mapa, llamamos a folium con las opciones que queramos. Por ejemplo, vamos a pintar un mapa de la zona de República Argentina, Madrid

In [7]:
folium.Map()

In [5]:
rep_argentina = [40.4438, -3.6857]
folium.Map(location=rep_argentina, zoom_start=15)

Las imágenes de las calles, el mapa base, se denominan `tiles`. Podemos personalizarlo con el parámetro `tiles`. Uno que suele quedar bastante bien para pintar datos encima es `cartodbpositron`:

In [6]:
folium.Map(location=rep_argentina, zoom_start=15, tiles='cartodbpositron') # El mapa base que utilizamos viene definido 
                                                                           # por el argumento tiles.

## Marcadores

Para añadir una localización, añadimos uno o varios marcadores al mapa. Sobre ellos, podemos personalizar:

* Localización
* Textos en tooltip o popup
* Estilo (forma, color, ...)

Mira más opciones en la [documentación](https://python-visualization.github.io/folium/modules.html#folium.map.Marker).

In [7]:
rep_argentina = [40.446, -3.68]
m = folium.Map(location=rep_argentina, zoom_start=15)

In [8]:
folium.Marker([40.4438, -3.6857], tooltip='EAE Joaquín Costa').add_to(m) 
folium.Marker([40.4485,-3.6796], tooltip='EAE Príncipe de Vergara').add_to(m)
m
# El acento podría dar problemas, pero en todo caso basta con quitarlo si pasa algo. Por lo pronto solo lo pone mal en el mapa.

Vamos a añadir una serie de localizaciones a nuestro mapa. En `dat/` tenemos disponibles las localizaciones de los apartamentos de AirBnB en Madrid, descargados de [Inside AirBnB](http://insideairbnb.com/).

In [9]:
listings = pd.read_csv('dat/listings.csv')
listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,6369,"Rooftop terrace room with ensuite bathroom, Airc.",13660,Simon,Chamartín,Hispanoamérica,40.45628,-3.67763,Private room,70,1,65,2019-09-04,0.56,1,53
1,21853,Bright and airy room,83531,Abdel,Latina,Cármenes,40.40341,-3.74084,Private room,17,4,33,2018-07-15,0.55,2,48
2,24805,Gran Via Studio Madrid,101471,Iraido,Centro,Universidad,40.42202,-3.70395,Entire home/apt,80,5,2,2017-07-03,0.03,1,354
3,24836,"Select the Madrid more ""cool"".",101653,Tenty,Centro,Justicia,40.41995,-3.69764,Entire home/apt,115,3,64,2019-07-07,0.63,1,301
4,26825,Single Room whith private Bathroom,114340,Agustina,Arganzuela,Legazpi,40.38985,-3.69011,Private room,25,2,136,2019-07-10,1.19,1,337


In [11]:
listings.shape # No le vamos a pintar los 20000, porque preferimos que el ordenador no pete.

(20837, 16)

In [12]:
# Mi mapa base sobre Madrid
map_madrid = folium.Map(location=[40.42, -3.7], zoom_start=13)

# Cojo 100 apartamentos (la muestra completa es muy grande)
listings_sample = listings.sample(100, random_state=1234)

# Los añado a mi mapa
# Voy a usar CircleMarker en lugar de Marker, porque me deja personalizar el color más fácilmente
listings_sample.apply(
    lambda row: folium.CircleMarker([row.latitude, row.longitude], tooltip=row.room_type, radius=3).add_to(map_madrid),
    axis=1
)
map_madrid

# Esta es la forma rebuscada de hacerlo, también se puede hacer con un for.

#### Ejercicio

Personaliza el mapa anterior para que el color del marcador diferencie por `room_type`.

In [19]:
room_types = listings_sample['room_type'].unique()
type(room_types)

numpy.ndarray

In [14]:
col_list = ['#ff99ff', '#ff33ff', '#0000ff','#000000' ]
color_dict = dict(zip(room_types, col_list))
color_dict.get('Hotel room') # Comprobamos que los valores se han enlazado correctamente.

'#000000'

In [22]:
my_map_madrid = folium.Map(location = [40.42,-3.7], zoom_start = 13)

for index,row in listings_sample.iterrows(): # Para iterar por las filas de un data.frame
    folium.CircleMarker([row['latitude'], row['longitude']], tooltip = row['room_type'], radius = 3, color = color_dict.get(row['room_type'])).add_to(my_map_madrid)
my_map_madrid

## Heatmaps

Son útiles para representar densidades.

Vamos a utilizarlo para ver cuál es la concentración de apartamentos por zona.

Si consultamos la [documentación](https://python-visualization.github.io/folium/plugins.html#folium.plugins.HeatMap), vemos que necesita un parámetro `data` que debe ser una lista de `[lat, lng]`, pero también acepta un dataframe de pandas con 2 columnas.

In [23]:
from folium.plugins import HeatMap

In [24]:
m = folium.Map(location=[40.42, -3.7], zoom_start=11, tiles='cartodbpositron')
heatmap = HeatMap(data=listings[["latitude", "longitude"]], radius=15).add_to(m)
m

## Mapa de coropletas

Son mapas sobre los que coloreamos las regiones que contiene en base a la propiedad que queremos explicar.

Vamos a pintar un mapa que represente el precio medio de los alojamientos de cada barrio.

In [30]:
num_per_area = listings.groupby('neighbourhood').id.count()
num_per_area # Es un pd Series.

neighbourhood
Abrantes         39
Acacias         198
Adelfas          87
Aeropuerto       16
Aguilas          49
               ... 
Valverde         81
Ventas          170
Vinateros        22
Vista Alegre    114
Zofío            48
Name: id, Length: 127, dtype: int64

In [28]:
num_per_area = listings.groupby('neighbourhood').id.count().reset_index()
num_per_area = num_per_area.rename(columns={"id": "num"})
num_per_area.head()

,neighbourhood,num
0,Abrantes,39
1,Acacias,198
2,Adelfas,87
3,Aeropuerto,16
4,Aguilas,49


In [26]:
# Incializamos el mapa
m = folium.Map(location=[40.42, -3.7], zoom_start=11, tiles='cartodbpositron')

# Add the color for the chloropleth:
folium.Choropleth(
    geo_data='dat/neighbourhoods.geojson', # Datos con las coordenadas de los habitantes.
    data=num_per_area, # Datos previamente filtrados con el nº de habitantes por vecindario.
    columns=['neighbourhood', 'num'], # Nombre de las columnas de num_per_area.
    key_on='feature.properties.neighbourhood', # va accediendo a través de las dimensiones del json hasta llegar a 'neighbourhood'.
    fill_color='YlGn' # Paleta de colores a usar.
).add_to(m)

m

#### Ejercicio

Modifica el mapa anterior para definir los límites de cada color y ver mejor el detalle. P.e. para que de 0 a 10 se vea en una intensidad, de 10 a 100 en otra, de 100 a 1000, ... (es decir, una escala logarítmica)